In [1]:
from loading import get_essay_sets, essays_by_set
import os
#import language_tool_python
import pandas as pd
# import spacy
# from spacy_readability import Readability

In [ ]:

def readabilty(essay):
    nlp = spacy.load('en')
    read = Readability(nlp)
    nlp.add_pipe(read, last=True)
    doc = nlp(essay)
    doc._.flesch_kincaid_grade_level
    doc._.flesch_kincaid_reading_ease
    doc._.dale_chall
    return doc._.flesch_kincaid_reading_ease

In [2]:
# lexical variety -> [0, 1] -> need to make a scale 0-2
def ttr(essay):
    words = essay.split()
    unique_words = set(words)
    return len(unique_words) / len(words)



In [3]:
def extract_features(essays, feature_functions):
    return [[f(es) for f in feature_functions] for es in essays]

In [8]:
test_path = path = os.path.join(os.getcwd(), 'data', 'public_leaderboard_rel_2.tsv')
feature_functions = [ttr, readabilty]
df = pd.DataFrame()
index = []
for essay_set in get_essay_sets():
        print("Making Features for Essay Set %s" % essay_set)
        # add essay set column and id
        train = list(essays_by_set(essay_set))
        #print(train[0])
        features = extract_features([x["EssayText"] for x in train], feature_functions)
        essay_set_df = pd.DataFrame(features, columns=['TTR', 'Grammar'])
        df = pd.concat([df, essay_set_df], ignore_index=True)
        idx = [int(x["Id"]) for x in train]
        index.extend(idx)
df['id']=index
output_filename = f'features.csv'
output_path = os.path.join(os.getcwd(), output_filename)
df.to_csv(output_path, index=False)  # Save without row index
print(f"Features saved to {output_path}")


Making Features for Essay Set 1.0
Making Features for Essay Set 2.0
Making Features for Essay Set 3.0
Making Features for Essay Set 4.0
Making Features for Essay Set 5.0
Making Features for Essay Set 6.0
Making Features for Essay Set 7.0
Making Features for Essay Set 8.0
Making Features for Essay Set 9.0
Making Features for Essay Set 10.0
Features saved to /Users/juliabelloni/Desktop/LLM/Automatic-SAS-with-LLMs/features.csv


In [ ]:
import language_tool_python
# grammar -> [0,1]
def grammar(essay):
    tool = language_tool_python.LanguageTool('en-US')
    test_essay = "This is a test."
    matches = tool.check(test_essay)
    print(matches)
    matches = tool.check(essay)
    sentences = essay.split(".")
    tool.close()
    return len(matches)/sentences
grammar("yap yap")